## Simulation of a M/M/1 queue using processes

## Common random numbers

In [1]:
using SimJulia
using Distributions
using RandomStreams
using Distributions

const SEED = 12345

rand_dist(rng::MRG32k3a, Dist::Distribution) = quantile(Dist, rand(rng))

seeds = [SEED, SEED, SEED, SEED, SEED, SEED]
gen = MRG32k3aGen(seeds)

Seed for next MRG32k3a generator:
[12345,12345,12345,12345,12345,12345]

In [2]:
include("tally.jl")

stdev (generic function with 1 method)

In [3]:
include("tallystore.jl")

add (generic function with 2 methods)

In [4]:
println("M/M/1 with processes")

M/M/1 with processes


In [5]:
#rates
λ = 1.9
μ = 2.0
ρ = λ/μ

0.95

In [6]:
type System
    W
    arrival
    service
    counter
    arrgen
    servgen
    
    function System(text::String)
        s = new()
        s.W = Tally(text)
        s.arrival = Exponential(1.0/λ)
        s.service = Exponential(1.0/μ)
        s.arrgen = next_stream(gen)
        s.servgen = next_stream(gen)
        return s;
    end
end

function restart(s::System)
    init(s.W)
    next_substream!(s.arrgen)
    next_substream!(s.servgen)
end

function system_set_arrival(s::System, rate::Float64)
    s.arrival = Exponential(1.0/rate)
end

function system_set_service(s::System, rate::Float64)
    s.service = Exponential(1.0/rate)
end

system_set_service (generic function with 1 method)

In [7]:
s = System("Waiting Times")

System(Tally("Waiting Times",0,0.0,Inf,-Inf,0.0,0.0,0.0),Distributions.Exponential{Float64}(θ=0.5263157894736842),Distributions.Exponential{Float64}(θ=0.5),#undef,Full state of MRG32k3a generator:
Cg = [12345,12345,12345,12345,12345,12345]
Bg = [12345,12345,12345,12345,12345,12345]
Ig = [12345,12345,12345,12345,12345,12345],Full state of MRG32k3a generator:
Cg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Bg = [3692455944,1366884236,2968912127,335948734,4161675175,475798818]
Ig = [3692455944,1366884236,2968912127,335948734,4161675175,475798818])

In [8]:
# Allow a simulation with a fixed number of clients
function source(env::Environment, s::System, limit::Float64)
    i = 0
    while (true)
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        if (now(env) > limit) break end
        i += 1
        Process(env, customer, s, i)
    end
end

function source(env::Environment, s::System, nCusts::Int64)
    for i = 1:nCusts
        yield(Timeout(env, rand_dist(s.arrgen, s.arrival)))
        Process(env, customer, s, i)
    end
end

function customer(env::Environment, s::System, idx::Int)
    # Record the arrival time in the system
    arrive = now(env)
    yield(Request(s.counter))
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    add(s.W, wait)
    yield(Timeout(env, rand_dist(s.servgen, s.service)))
    yield(Release(s.counter))
end

customer (generic function with 1 method)

In [9]:
function onesim(s::System)
    nCustomers = 4000
    
    env = Environment()
    s.counter = Resource(env, 1)
    
    Process(env, source, s, nCustomers)
    run(env)
end

onesim (generic function with 1 method)

In [10]:
meanWaits = TallyStore("Average waiting time")

TallyStore(Float64[],Tally("Average waiting time",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [11]:
nSims = 100

100

In [12]:
function simulation(s::System, mw::TallyStore)
    for n=1:nSims
        restart(s)
        onesim(s)
        w = average(s.W)
        N = nobs(s.W)
        add(mw, w)
        println("Sim $n. Average waiting time: $w. Number of observations: $N")
    end
end

simulation (generic function with 1 method)

In [13]:
simulation(s, meanWaits)

Sim 1. Average waiting time: 8.063337078141043. Number of observations: 4000
Sim 2. Average waiting time: 5.2417854667011206. Number of observations: 4000
Sim 3. Average waiting time: 7.6017913198944935. Number of observations: 4000
Sim 4. Average waiting time: 6.296363224961417. Number of observations: 4000
Sim 5. Average waiting time: 8.095683340399296. Number of observations: 4000
Sim 6. Average waiting time: 4.903988427782636. Number of observations: 4000
Sim 7. Average waiting time: 6.833001851372892. Number of observations: 4000
Sim 8. Average waiting time: 9.92720878209105. Number of observations: 4000
Sim 9. Average waiting time: 3.805249867425339. Number of observations: 4000
Sim 10. Average waiting time: 5.639068201593208. Number of observations: 4000
Sim 11. Average waiting time: 11.1225862541573. Number of observations: 4000
Sim 12. Average waiting time: 12.078229676858033. Number of observations: 4000
Sim 13. Average waiting time: 6.324130693190484. Number of observations:

In [14]:
function ci_normal(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(Normal(), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_normal (generic function with 1 method)

In [15]:
ci_normal(nobs(meanWaits.t), average(meanWaits.t), stdev(meanWaits.t), 0.05)

(8.283602799795442,9.955826736999832)

In [16]:
function ci_tdist(n::Int64, mean::Float64, stdev::Float64, α::Float64)
    z = quantile(TDist(n-1), 1-α/2)
    w = z*stdev/sqrt(n)
    # Lower bound
    l = mean - w
    # Upper bound
    u = mean + w
    
    return l, u
end

ci_tdist (generic function with 1 method)

In [17]:
ci_tdist(nobs(meanWaits.t)-1, average(meanWaits.t), stdev(meanWaits.t), 0.05)

(8.268884889585166,9.970544647210108)

Average waiting time according to M/M/1 formulas. In stationnary regime, the mean waiting time is
$$ \overline{W} = \frac{\frac{\rho}{\mu}}{1-\rho} = \frac{\rho}{\mu-\lambda}. $$

In [18]:
w = (ρ/μ)/(1-ρ)

9.499999999999991

In [19]:
meanWaitsFast = TallyStore("Average waiting time in improved system")

TallyStore(Float64[],Tally("Average waiting time in improved system",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [20]:
μFast = 2.05
ρFast= λ/μFast

wFast = (ρFast/μFast)/(1-ρFast)

6.1788617886178905

In [21]:
system_set_service(s, μFast)

Distributions.Exponential{Float64}(θ=0.48780487804878053)

In [22]:
simulation(s, meanWaitsFast)

Sim 1. Average waiting time: 4.279759918860097. Number of observations: 4000
Sim 2. Average waiting time: 10.177509503512976. Number of observations: 4000
Sim 3. Average waiting time: 4.168677522108386. Number of observations: 4000
Sim 4. Average waiting time: 5.017804311514114. Number of observations: 4000
Sim 5. Average waiting time: 7.342214473335145. Number of observations: 4000
Sim 6. Average waiting time: 13.102297909365122. Number of observations: 4000
Sim 7. Average waiting time: 3.558567111908596. Number of observations: 4000
Sim 8. Average waiting time: 4.909200629679188. Number of observations: 4000
Sim 9. Average waiting time: 3.9430384478510936. Number of observations: 4000
Sim 10. Average waiting time: 5.583225511302099. Number of observations: 4000
Sim 11. Average waiting time: 8.280104057604865. Number of observations: 4000
Sim 12. Average waiting time: 7.279173222990389. Number of observations: 4000
Sim 13. Average waiting time: 6.579745395803486. Number of observation

In [23]:
Δ = Tally("Difference of systems")

for i = 1:nSims
    add(Δ, meanWaits.obs[i]-meanWaitsFast.obs[i])
end

ci_tdist(nobs(Δ)-1, average(Δ), stdev(Δ), 0.05)

(2.19847027874309,4.234841513960088)

In [24]:
cor(meanWaits.obs, meanWaitsFast.obs)

-0.10660603889619878

In [25]:
reset_stream!(s.arrgen)
reset_stream!(s.servgen)

meanWaitsFast = TallyStore("Average waiting time in improved system")

TallyStore(Float64[],Tally("Average waiting time in improved system",0,0.0,Inf,-Inf,0.0,0.0,0.0))

In [26]:
simulation(s, meanWaitsFast)

Sim 1. Average waiting time: 5.74341262357014. Number of observations: 4000
Sim 2. Average waiting time: 3.956450123015612. Number of observations: 4000
Sim 3. Average waiting time: 4.757521445441934. Number of observations: 4000
Sim 4. Average waiting time: 4.40488142615353. Number of observations: 4000
Sim 5. Average waiting time: 5.911760171993039. Number of observations: 4000
Sim 6. Average waiting time: 3.881501639774835. Number of observations: 4000
Sim 7. Average waiting time: 5.022509396530369. Number of observations: 4000
Sim 8. Average waiting time: 6.972364146977035. Number of observations: 4000
Sim 9. Average waiting time: 2.9996531621157527. Number of observations: 4000
Sim 10. Average waiting time: 4.229132771032018. Number of observations: 4000
Sim 11. Average waiting time: 8.050200999794447. Number of observations: 4000
Sim 12. Average waiting time: 7.092506368166094. Number of observations: 4000
Sim 13. Average waiting time: 4.558642572614625. Number of observations: 4

In [27]:
Δ = Tally("Difference of systems")

for i = 1:nSims
    add(Δ, meanWaits.obs[i]-meanWaitsFast.obs[i])
end

ci_tdist(nobs(Δ)-1, average(Δ), stdev(Δ), 0.05)

(2.487136895265628,3.464719290131804)

In [28]:
cor(meanWaits.obs, meanWaitsFast.obs)

0.9271227028850821

## Antithetic variates

In [29]:
function source_av(env::Environment, s::System, limit::Float64)
    i = 0
    while (true)
        u = rand(s.arrgen)
        if (arrival_av)
            u = 1-u
        end
        yield(Timeout(env, quantile(s.arrival, u)))
        if (now(env) > limit) break end
        i += 1
        Process(env, customer_av, s, i)
    end
end

function source_av(env::Environment, s::System, nCusts::Int64)
    for i = 1:nCusts
        u = rand(s.arrgen)
        if (arrival_av)
            u = 1-u
        end
        yield(Timeout(env, quantile(s.arrival, u)))
        Process(env, customer_av, s, i)
    end
end

function customer_av(env::Environment, s::System, idx::Int)
    # Record the arrival time in the system
    arrive = now(env)
    yield(Request(s.counter))
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    add(s.W, wait)
    u = rand(s.servgen)
    if (service_av)
        u = 1-u
    end
    yield(Timeout(env, quantile(s.service, u)))
    yield(Release(s.counter))
end

customer_av (generic function with 1 method)

In [30]:
function onesim_av(s::System)
    nCustomers = 10000
    
    env = Environment()
    s.counter = Resource(env, 1)
    
    Process(env, source_av, s, nCustomers)
    run(env)
end

onesim_av (generic function with 1 method)

In [31]:
function simulation_av(s::System, mw::TallyStore)
    for n=1:nSims
        restart(s)
        onesim_av(s)
        w = average(s.W)
        N = nobs(s.W)
        add(mw, w)
        println("Sim $n. Average waiting time: $w. Number of observations: $N")
    end
end

simulation_av (generic function with 1 method)

In [32]:
reset_stream!(s.arrgen)
reset_stream!(s.servgen)

system_set_service(s, μ)

arrival_av = false
service_av = false

nSims = 100

meanWaits = TallyStore("Average waiting time")

simulation_av(s, meanWaits)

Sim 1. Average waiting time: 7.550875546475624. Number of observations: 10000
Sim 2. Average waiting time: 6.135640065836502. Number of observations: 10000
Sim 3. Average waiting time: 18.479700395704146. Number of observations: 10000
Sim 4. Average waiting time: 7.1030035604970845. Number of observations: 10000
Sim 5. Average waiting time: 8.403267787576848. Number of observations: 10000
Sim 6. Average waiting time: 6.415659416467969. Number of observations: 10000
Sim 7. Average waiting time: 8.73956120790153. Number of observations: 10000
Sim 8. Average waiting time: 13.827590833432248. Number of observations: 10000
Sim 9. Average waiting time: 7.2529369795613725. Number of observations: 10000
Sim 10. Average waiting time: 6.972471705468727. Number of observations: 10000
Sim 11. Average waiting time: 25.000757727815806. Number of observations: 10000
Sim 12. Average waiting time: 10.403505705804962. Number of observations: 10000
Sim 13. Average waiting time: 6.573875031066891. Number 

In [33]:
ci_tdist(nobs(meanWaits.t)-1, average(meanWaits.t), stdev(meanWaits.t), 0.05)

(8.729965100548558,10.240166430376991)

In [34]:
reset_stream!(s.arrgen)
#reset_stream!(s.servgen)

arrival_av = true
service_av = false

meanWaitsAV = TallyStore("Average waiting time - AV")

simulation_av(s, meanWaitsAV)

Sim 1. Average waiting time: 7.9813040523646075. Number of observations: 10000
Sim 2. Average waiting time: 14.155589226275314. Number of observations: 10000
Sim 3. Average waiting time: 5.907469949839986. Number of observations: 10000
Sim 4. Average waiting time: 6.948451581724155. Number of observations: 10000
Sim 5. Average waiting time: 7.0310024711250785. Number of observations: 10000
Sim 6. Average waiting time: 13.013587922161921. Number of observations: 10000
Sim 7. Average waiting time: 6.878407342029873. Number of observations: 10000
Sim 8. Average waiting time: 7.242959742501776. Number of observations: 10000
Sim 9. Average waiting time: 10.489788804894689. Number of observations: 10000
Sim 10. Average waiting time: 14.490326217652342. Number of observations: 10000
Sim 11. Average waiting time: 9.430398719949842. Number of observations: 10000
Sim 12. Average waiting time: 7.548676492874876. Number of observations: 10000
Sim 13. Average waiting time: 11.178156439000968. Numbe

In [35]:
average(meanWaits.t)

9.485065765462775

In [36]:
average(meanWaitsAV.t)

9.166518540021636

In [37]:
mean = (average(meanWaits.t)+average(meanWaitsAV.t))/2

9.325792152742206

In [38]:
variance(meanWaits.t)

14.333654709875363

In [39]:
variance(meanWaitsAV.t)

8.263154246841559

In [40]:
cov(meanWaits.obs, meanWaitsAV.obs)

-2.421995444931172

In [41]:
cor(meanWaits.obs, meanWaitsAV.obs)

-0.2225470915995911

In [42]:
σ2 = (variance(meanWaits.t)+variance(meanWaitsAV.t)+2*cov(meanWaits.obs, meanWaitsAV.obs))/4 

4.438204516713644

In [43]:
ci_tdist(2*nSims-1, mean, sqrt(σ2), 0.05)

(9.031290404604338,9.620293900880075)